In [1]:
from synthesizeme.personalrm.synthesizeme import SynthesizeMe
from synthesizeme.datasets import PrismDataset

/opt/miniconda3/envs/personal_rm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Collect User Feedback

In [2]:
dataset = PrismDataset()
train, val, test = dataset.get_user_data(dataset.get_user_ids()[12])

In [3]:
data = train[:]
data.extend(val)

## View the data

In [4]:
data[0]

{'conversation_id': 'c2596',
 'user_id': 'user488',
 'context': array([{'content': 'Hi, how are you?', 'role': 'user'}], dtype=object),
 'chosen': {'content': "Hello! I'm here and ready to assist you. How can I help you today?",
  'role': 'model'},
 'rejected': {'content': "I'm doing great, thank you for asking! I'm here to assist you in our conversation. How may I help you today?",
  'role': 'model'},
 'turns': 1,
 'type': 'unguided',
 'split': 'train',
 'flip': False}

## Train Personal Reward Model

In [5]:
# rm = SynthesizeMe(model_id='together_ai/meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8', num_workers=64)
rm = SynthesizeMe(model_id='gemini/gemini-2.0-flash', num_workers=64)

In [6]:
rm.fit(data)

2025/05/22 00:34:57 WARNING dspy.primitives.module: There is a mismatch of python version between saved model and current environment. You saved with `python==3.12`, but now you have `python==3.10`. This might cause errors or performance downgrade on the loaded model, please consider loading the model in the same environment as the saving environment.
2025/05/22 00:34:57 WARNING dspy.primitives.module: There is a mismatch of dspy version between saved model and current environment. You saved with `dspy==2.6.12`, but now you have `dspy==2.6.24`. This might cause errors or performance downgrade on the loaded model, please consider loading the model in the same environment as the saving environment.


Going to sample between 1 and 16 traces per predictor.
Will attempt to bootstrap 10 candidate sets.



  0%|          | 0/16 [00:00<?, ?it/s]













































 19%|█▉        | 3/16 [00:00<00:00, 41.42it/s]

Bootstrapped 2 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.






















...

2025/05/22 00:34:58 INFO dspy.evaluate.evaluate: Average Metric: 6 / 8 (75.0%)



Average Metric: 4.00 / 8 (50.0%): 100%|██████████| 8/8 [00:00<00:00, 183.68it/s]

2025/05/22 00:34:58 INFO dspy.evaluate.evaluate: Average Metric: 4 / 8 (50.0%)



New best score: 37.5 for seed -3
Scores so far: [37.5]
Best score so far: 37.5
Scores so far: [37.5, 37.5]
Best score so far: 37.5
New best score: 75.0 for seed -1
Scores so far: [37.5, 37.5, 75.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0, 75.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0, 75.0, 50.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0, 75.0, 50.0, 62.5]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0, 75.0, 50.0, 62.5, 75.0]
Best score so far: 75.0
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0, 75.0, 50.0, 62.5, 75.0, 62.5]
Best score so far: 75.0
New best score: 87.5 for seed 8
Scores so far: [37.5, 37.5, 75.0, 75.0, 50.0, 25.0, 75.0, 50.0, 6

judge_persona.predict = Predict(StringSignature(conversation, first_completion, second_completion -> reasoning, preference
    instructions="Given a conversation and two completions from different models, alongside some prior judgements and a user persona, determine which completion the human judge is more likely to prefer.  Use any provided context as well as the provided persona to speculate about the personal preferences of the judge.  You are a personalized reward model for this user, so think carefully about what this user will like.\nThe user you are judging completions for has the FOLLOWING PERSONA: ===\nThe synthesized persona is that of a thoughtful, inquisitive, and ethically-minded individual who seeks comprehensive and nuanced information to form their own informed opinions. They are not satisfied with simplistic answers or superficial explanations. They value depth, complexity, and a balanced perspective. They are also action-oriented and appreciate practical advice that c

In [7]:
rm.get_persona()

'The synthesized persona is that of a thoughtful, inquisitive, and ethically-minded individual who seeks comprehensive and nuanced information to form their own informed opinions. They are not satisfied with simplistic answers or superficial explanations. They value depth, complexity, and a balanced perspective. They are also action-oriented and appreciate practical advice that can be applied to real-life situations. This persona is likely someone who is engaged in social issues, values ethical considerations, and strives to understand the complexities of the world around them. They are open-minded and respectful of different viewpoints, but they also have a strong sense of their own values and principles. They are likely well-read, intellectually curious, and committed to making a positive impact on the world.'

In [8]:
rm.get_demos()[0]

{'conversation': '[user]: Do you agree with death penalty?\n\n[model]: I\'m just an AI, I don\'t have personal opinions or beliefs, and I cannot endorse or support any form of violence or punishment, including the death penalty. It is important to respect the inherent value of human life and to find alternative, humane ways to address criminal behavior and provide justice for victims and their families.\n\n[user]: What do you think of people that agree with death penalty?\n\n[model]: I\'m just an AI, I don\'t have personal opinions or beliefs, and I cannot endorse or support any form of violence or punishment, including the death penalty. It is important to respect the inherent value of human life and to find alternative, humane ways to address criminal behavior and provide justice for victims and their families.\n\nIt is understandable that some people may feel that the death penalty is a necessary tool for deterring crime and providing justice for victims, but it is important to cons

In [9]:
rm.evaluate(test)

{'mean': 1.0, 'lower_bound': 1.0, 'upper_bound': 1.0}